# import module

In [ ]:
import numpy as np
import numba
import xarray as xr

# Continuous Equation

$$
    \begin{align}
        \frac{\partial h}{\partial t}+\frac{\partial q_x}{\partial x} +\frac{\partial q_y}{\partial y} = 0
    \end{align}
$$

In [ ]:
@numba.jit(nopython=True, parallel=False)
def conEq(dep, qx, qy, zb, dt, dx, dy, hmin, hbuf):
    imax, jmax = len(dep), len(dep[0])
    depn = np.empty_like(dep)
    fluxx = np.zeros((imax+1, jmax))
    fluxy = np.zeros((imax, jmax+1))
    gravity = 9.8
    
    f = lambda Qp, Qm : Qm if Qp >= 0.0 and Qm >= 0.0 else (Qp if Qp <= 0.0 and Qm <= 0.0 else 0.5*Qp+0.5*Qm )
    
    def flux(Qp, Qm, depp, depm, zbp, zbm) : 
        r = f(Qp, Qm)
        
        if ( (depm + zbm) < zbp ) and (depp <= hbuf) : r = 0.0
        if ( (depp + zbp) < zbm ) and (depm <= hbuf) : r = 0.0
            
        return r
        
    for i in numba.prange( 1, imax ):
        for j in numba.prange( jmax ):
            c, xm = (i,j), (i-1,j)
            fluxx[c] = flux(qx[c], qx[xm], dep[c], dep[xm], zb[c], zb[xm])
            
# boundary : not use
    fluxx[-1,:] = -9999
    fluxx[0,:] = -9999
        
    for i in numba.prange( imax ):
        for j in numba.prange( 1, jmax ):
            c, ym = (i,j), (i,j-1)
            fluxy[c] = flux(qy[c], qy[ym], dep[c], dep[ym], zb[c], zb[ym])
            
# wall boundary 
    fluxy[:,0] = 0.0 #fluxy[:,0]
    fluxy[:,-1] = 0.0 #fluxy[:,0]
    
    n = 0
    for i in numba.prange(1, imax):
        for j in numba.prange(jmax):
            c, xp, yp = (i, j), (i+1, j), (i, j+1)
            depn[c] = dep[c] - dt*(fluxx[xp] - fluxx[c])/dx - dt*(fluxy[yp] - fluxy[c])/dy
            if depn[c] < hmin : 
                n += 1
                depn[c] = hmin
                
# upstream boundary                
    depn[0][:] = depn[1][:]
    
# downstream boundary 
    depn[-1][:] = depn[-2][:]
    
    return depn, n

# Momentum Equation

$$
    \begin{align}
        \frac{\partial q_x}{\partial t}+\frac{\partial u q_x}{\partial x}+\frac{\partial v q_x}{\partial y}+gh\frac{\partial H}{\partial x}+\frac{\tau_{0x}}{\rho} 
        - \nu_t h \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right)= 0 \\
        \frac{\partial q_y}{\partial t}+\frac{\partial u q_y}{\partial x}+\frac{\partial v q_y}{\partial y}+gh\frac{\partial H}{\partial y}+\frac{\tau_{0y}}{\rho}- \nu_t h \left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2} \right)
        = 0
    \end{align}
$$

In [ ]:
@numba.jit(nopython=True, parallel=False)
def momentEq(dep, qx, qy, depn, zb, Qup, dt, dx, dy, hmin, hbuf, direction):
    #direction = 1:x, 2:y
    gravity = 9.8
    cManing = 0.03

    q = qx if direction == 1 else qy
    u, v = qx/dep, qy/dep
    Vdir = q/dep  
        
    imax, jmax = len(q), len(q[0])
    
    qn = np.empty_like(q)
    fluxx = np.zeros((imax+1, jmax))
    fluxy = np.zeros((imax, jmax+1))
    
#     f = lambda vp,vm,qp,qm : vm*qm if vp > 0.0 and vm > 0.0 else \
#         (vp*qp if vp < 0.0 and vm < 0.0 else (0.5*vp+0.5*vm)*(0.5*qp+0.5*qm) )
        
    f = lambda vp,vm,qp,qm : vm*qm if vp >= 0.0 and vm >= 0.0 else \
        (vp*qp if vp <= 0.0 and vm <= 0.0 else (0.5*vp+0.5*vm)*(0.5*qp+0.5*qm) )
        
    def flux1(vp, vm, qp, qm, depp, depm, zbp, zbm) : 
        if ( (depm + zbm) < zbp ) and (depp <= hbuf) : r = 0.0
        if ( (depp + zbp) < zbm ) and (depm <= hbuf) : r = 0.0
            
        r = f(vp,vm,qp,qm)
        return r
    
    for i in numba.prange( 1, imax ):
        for j in numba.prange( jmax ):
            c, xm = (i,j), (i-1,j)
            fluxx[c] = flux1( u[c], u[xm], q[c], q[xm], dep[c], dep[xm], zb[c], zb[xm] )
            
# boundary : not use
    fluxx[-1,:] = -9999
    fluxx[0,:]  = -9999

    for i in numba.prange( imax ):
        for j in numba.prange( 1, jmax ):
            c, ym = (i,j), (i,j-1)
            fluxy[c] = flux1( v[c], v[ym], q[c], q[ym], dep[c], dep[ym], zb[c], zb[ym] )
            
# wall boundary 
    fluxy[:,0]  = -fluxy[:,1]
    fluxy[:,-1] = -fluxy[:,-2]
    
    for i in numba.prange(1, imax-1):
        for j in numba.prange(jmax):    
            c = (i, j)
            if depn[c] <= hbuf :
                qn[c] = 0.0
            else:
            # pressure & gravity term
                if direction == 2 and ((j == 0) or (j == jmax-1)) : 
                    if j == 0 :
                        c, yp = (i, j), (i, j+1)
                        Hc, Hp = depn[c]+zb[c], depn[yp]+zb[yp]
                        
                        if Hc < zb[yp] and depn[yp] <= hbuf :
                            dHdx = 0.0
                        else :
                            dHdx = ( Hp - Hc )/dy
                    elif j == jmax-1 :
                        c, ym = (i, j), (i, j-1)
                        Hc, Hm = depn[c]+zb[c], depn[ym]+zb[ym]
                        
                        if Hc < zb[ym] and depn[ym] <= hbuf :
                            dHdx = 0.0
                        else :
                            dHdx = ( Hc - Hm )/dy
                            
                elif direction == 1 and i == imax-1 :
                    pass
                else :
                    c  = (i, j)
                    if direction == 1 :
                        xp = (i+1, j)
                        xm = (i-1, j)
                        dp, dm, delta, dib = xp, xm, dx, 0.0
                    else :
                        yp = (i, j+1)
                        ym = (i, j-1)
                        dp, dm, delta, dib = yp, ym, dy, 0.0
                        
                    Vc, Vp, Vm = q[c]/dep[c], q[dp]/dep[dp], q[dm]/dep[dm]
                    Hc, Hp, Hm = depn[c]+zb[c], depn[dp]+zb[dp], depn[dm]+zb[dm]
                        
                    if Hc < zb[dp] and depn[dp] <= hbuf :
                        if Hc < zb[dm] and depn[dm] <= hbuf :
                            dHdx = 0.0
                        else:
                            dHdx = (Hc-Hm)/delta
                    elif Hc < zb[dm] and depn[dm] <= hbuf :
                        dHdx = (Hp-Hc)/delta
                    else :
                        if Vc > 0.0 and Vp > 0.0 and Vm > 0.0: 
                            Cr1, Cr2 = 0.5*(abs(Vc)+abs(Vp))*dt/delta, 0.5*(abs(Vc)+abs(Vm))*dt/delta
                            dHdx1, dHdx2 = (Hp-Hc)/delta-dib, (Hc-Hm)/delta-dib
                        elif Vc < 0.0 and Vp < 0.0 and Vm < 0.0:
                            Cr1, Cr2 = 0.5*(abs(Vc)+abs(Vm))*dt/delta, 0.5*(abs(Vc)+abs(Vp))*dt/delta
                            dHdx1, dHdx2 = (Hc-Hm)/delta-dib, (Hp-Hc)/delta-dib          
                        else:
                            Cr1 = Cr2 = 0.5*(abs(0.5*(Vc+Vp))+abs(0.5*(Vc+Vm)))*dt/delta
                            dHdx1 = dHdx2 = (0.5*(Hc+Hp) - 0.5*(Hc+Hm)) / delta - dib
            
                        w1, w2 = 1-Cr1**0.5, Cr2**0.5
                        dHdx = w1 * dHdx1 + w2 * dHdx2   
                
# viscous sublayer
                Cf = gravity*cManing**2.0/dep[c]**(1.0/3.0) 
                Vnorm = np.sqrt(u[c]**2.0+v[c]**2.0) 
                Vis = Cf * Vnorm * u[c] if direction == 1 else  Cf * Vnorm * v[c]
            
# turbulence
#                kenergy = 2.07*Cf*Vnorm**2
                nut = 0.4/6.0*dep[c]*np.sqrt(Cf)*np.abs(Vnorm)
    
# side boundary : non-slip condition
                if i == imax-1: 
                    turb = 0.0
                elif j == 0 : 
                    if direction == 1:
                        turb = nut * ( Vdir[xp] - 2.0*Vdir[c] + Vdir[xm] )/ dx**2 \
                             + nut * ( Vdir[yp] - 3.0*Vdir[c] )/ dy**2
                    elif direction == 2:
                        turb = nut * ( Vdir[xp] - 2.0*Vdir[c] + Vdir[xm] )/ dx**2 \
                             + nut * ( Vdir[yp] - 3.0*Vdir[c] )/ dy**2
                elif j == jmax-1 : 
                    if direction == 1:
                        turb = nut * ( Vdir[xp] - 2.0*Vdir[c] + Vdir[xm] )/ dx**2 \
                             + nut * (  - 3.0*Vdir[c] + Vdir[ym] )/ dy**2
                    elif direction == 2:
                        turb = nut * ( Vdir[xp] - 2.0*Vdir[c] + Vdir[xm] )/ dx**2 \
                             + nut * (  - 3.0*Vdir[c] + Vdir[ym] )/ dy**2
                else :
                    turb = nut * ( Vdir[xp] - 2.0*Vdir[c] + Vdir[xm] )/ dx**2 \
                         + nut * ( Vdir[yp] - 2.0*Vdir[c] + Vdir[ym] )/ dy**2
                        
                c, xp, yp = (i,j), (i+1,j), (i,j+1)
                sourcet = Vis - dep[c] * turb
                
                qn[c] = q[c] - dt * ( fluxx[xp] - fluxx[c] ) / dx \
                             - dt * ( fluxy[yp] - fluxy[c] ) / dy \
                             - dt * gravity * depn[c] * dHdx \
                             - dt * sourcet
                            
# upstream boundary
    if direction == 2 : 
        qn[0,:] = 0.0
    else :
        updep = depn.copy()
        for j in range(jmax):
            updep[0,j] = 0.0 if updep[0,j] <= hmin else updep[0,j]
            
        alpha = Qup / dy / np.sum( updep[0,:]**(5/3) )
        qn[0,:] = alpha * updep[0,:]**(5/3)
        
# downstream boundary
    qn[-1,:] = qn[-2,:]
        
    return qn

In [ ]:
@numba.jit(nopython=True, parallel=False)
def simulation(dep, qx, qy, zb, Qup, dt, dx, dy, hmin, hbuf):
    depn, count = conEq(dep, qx, qy, zb, dt, dx, dy, hmin, hbuf)
    qxn = momentEq(dep, qx, qy, depn, zb, Qup, dt, dx, dy, hmin, hbuf, 1)
    qyn = momentEq(dep, qx, qy, depn, zb, Qup, dt, dx, dy, hmin, hbuf, 2)
    CFL = ((np.abs(qxn/depn))/dx + (np.abs(qyn/depn))/dy)*dt
    CFLmin = np.max(CFL)
    return depn, qxn, qyn, CFLmin, count

# bed variation

$$
    \begin{align}
       (1-\lambda) \frac{\partial z_b}{\partial t}+\frac{\partial q_{bx}}{\partial x} +\frac{\partial q_{by}}{\partial y} = 0 \\
        q_b : M.P.M.-formula
    \end{align}
$$

In [ ]:
@numba.jit(nopython=True, parallel=False)
def bedvarEq(dzb, dep, qx, qy, zb, dt, dx, dy, hmin, hbuf):
    imax, jmax = len(dep), len(dep[0])
    dzbn = np.empty_like(dep)
    qbx = np.empty_like(dep)
    qby = np.empty_like(dep)
    fluxx = np.zeros((imax+1, jmax))
    fluxy = np.zeros((imax, jmax+1))
    gravity = 9.8
    cManning = 0.03
    rhosw = 1.65
    porosity = 0.4
    di = 5.0 / 1000.0
    tsc = 0.05
    
    f = lambda Qp, Qm : Qm if Qp >= 0.0 and Qm >= 0.0 else (Qp if Qp <= 0.0 and Qm <= 0.0 else 0.5*Qp+0.5*Qm )
    
    def flux(Qp, Qm, depp, depm, zbp, zbm) : 
        r = f(Qp, Qm)
        
        # Dry bed
        if ( (depm + zbm) < zbp ) and (depp <= hbuf) : r = 0.0
        if ( (depp + zbp) < zbm ) and (depm <= hbuf) : r = 0.0
            
        return r
        
    for i in numba.prange(imax):
        for j in numba.prange(jmax):
            c = (i, j)
            u = qx[c]/dep[c]
            v = qy[c]/dep[c]
            Cf = gravity * cManning**2.0 / dep[c]**(1.0/3.0) 
            Vnorm = np.sqrt( u**2 + v**2 )
            us = Vnorm*np.sqrt(Cf)
            ts = us**2.0/rhosw/gravity/di 
            
            if ts/tsc > 1.0 :
            # MPM
                qb = np.sqrt(rhosw*gravity*di**3.0) * 8.0*ts**1.5 * (1.0 - tsc/ts)**1.5
                qbx[c] = qb * u/Vnorm 
                qby[c] = qb * v/Vnorm 
            else :
                qbx[c] = 0.0 
                qby[c] = 0.0 
    
    
    for i in numba.prange( 1, imax ):
        for j in numba.prange( jmax ):
            c, xm = (i,j), (i-1,j)
            fluxx[c] = flux(qbx[c], qbx[xm], dep[c], dep[xm], zb[c]+dzb[c], zb[xm]+dzb[xm])
            
# boundary 
    fluxx[-1,:] = qbx[-1, :]
    fluxx[0,:] = fluxx[1,:] # equilibrium supply
        
    for i in numba.prange( imax ):
        for j in numba.prange( 1, jmax ):
            c, ym = (i,j), (i,j-1)
            fluxy[c] = flux(qby[c], qby[ym], dep[c], dep[ym], zb[c]+dzb[c], zb[ym]+dzb[ym])
            
# wall boundary 
    fluxy[:,0] = 0.0 
    fluxy[:,-1] = 0.0 
    
    for i in numba.prange(imax):
        for j in numba.prange(jmax):
            c, xp, yp = (i, j), (i+1, j), (i, j+1)
            dzbn[c] = dzb[c] - dt/(1-porosity)*(fluxx[xp] - fluxx[c])/dx - dt/(1-porosity)*(fluxy[yp] - fluxy[c])/dy
                
# upstream boundary                
    dzbn[0,:] = dzbn[1,:]

    return dzbn, qbx, qby

# Main routine

In [ ]:
def d2dmodel(dep, qx, qy, zb, dzb, Qup, tmax, trunup):
    hmin = 10.0**(-5) 
    hbuf = 10.0**(-2) 
#     dzb = np.zeros_like(dep)
    qbx = np.zeros_like(dep)
    qby = np.zeros_like(dep)
    nxmax = len(zb)
    nymax = len(zb[0])
    dx = 5.0
    dy = 5.0
    dt = 0.02
    dtout= 5*3600.0
    irunup = -int(trunup/dt)
    
    def timeGenerater():
        it, itout = irunup, 0
        isTiter, isOut = True, False
        isSedimentcal = False
        yield it*dt, isTiter, isOut, isSedimentcal
    
        while True:
            it += 1
            if it*dt >= itout*dtout:
                itout += 1
                isOut = True
            else:
                isOut = False
    
            if it*dt >= tmax: isTiter = False
            if it > 0 : isSedimentcal = True 
    
            yield it*dt, isTiter, isOut, isSedimentcal
            
    tgen = timeGenerater() 
    t, isTiter, isOut, isSedimentcal = next(tgen)
    
    tarr = []
    deparr = np.empty((0,nxmax,nymax))
    uarr   = np.empty((0,nxmax,nymax)) 
    varr   = np.empty((0,nxmax,nymax))
    dzbarr = np.empty((0,nxmax,nymax))
    
    nout= 1
    while isTiter:
        t, isTiter, isOut, isSedimentcal = next(tgen)
        
        dep, qx, qy, CFLmin, count = simulation(dep, qx, qy, zb+dzb, Qup, dt, dx, dy, hmin, hbuf)
        if isSedimentcal : dzb, qbx, qby = bedvarEq(dzb, dep, qx, qy, zb, dt, dx, dy, hmin, hbuf)
        
        if isOut:
            tarr.append(round(t,2))
            deparr = np.append(deparr, np.array([dep]), axis=0)
            uarr = np.append(uarr, np.array([qx/dep]), axis=0)
            varr = np.append(varr, np.array([qy/dep]), axis=0)
            dzbarr = np.append(dzbarr, np.array([dzb]), axis=0)
            print(t, CFLmin, count)
            nout += 1
            dss = xr.Dataset( { 'depth': (['t','x','y'], deparr), 'u': (['t','x','y'], uarr), 'v': (['t','x','y'], varr)
                               , 'dzb': (['t','x','y'], dzbarr) }
                             , coords={'x': np.arange(nxmax)*dx + 0.5*dx , 'y':np.arange(nymax)*dy + 0.5*dy, 't': tarr} )
            
    dss.to_netcdf('output.nc')
    dss.close()
    return dep, qx, qy, dzb, qbx, qby

# calculation

In [ ]:
%%time
dx = 5
hmin = 10.0**(-5) 

nxmax, nymax = 200, 20
zb = np.zeros((nxmax, nymax))
qx = np.zeros((nxmax,nymax))
qy = np.zeros_like(qx)

ib = 0.001
Hs = 1.0
for i in range(nxmax):
    zb[i,:] = Hs - ib * i * dx

zb[:60, :5]  += 5.0 
zb[:60, 15:] += 5.0 
    
dep = np.full_like(zb, 1.0)
dep[:60, :5]  = hmin
dep[:60, 15:] = hmin

qy = np.full_like(zb, 0.0) 
qx = 1/0.03*ib**0.5*dep**(5/3) 

qx[:60, :5]  = 0.0
qx[:60, 15:] = 0.0
Qup = np.sum(qx[0])*dx

dep, qx, qy, dzb, qbx, qby = d2dmodel(dep, qx, qy, zb, dzb, Qup, tmax=500*3600, trunup=3600) 